In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "auto"
model_path = "ibm-granite/granite-3.0-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# drop device_map if running on CPU
model = AutoModelForCausalLM.from_pretrained(model_path, device_map=device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [2]:
# DATASET
from datasets import load_dataset, Dataset

## Load the dataset
ds = load_dataset("UWV/Leesplank_NL_wikipedia_simplifications_preprocessed", split='test')
df = ds.to_pandas()
# ds = ds.shuffle(seed=42)

# random_row = [i for i in ds.take(1)][0]

In [ ]:
# system_prompt = "Vereenvoudig een Nederlandse alinea tot één duidelijke en aantrekkelijke tekst, geschikt voor volwassenen die Nederlands als tweede taal spreken, met woorden uit de 'basiswoordenlijst Amsterdamse kleuters.' Behoud directe citaten, maak dialogen eenvoudiger en leg culturele verwijzingen, uitdrukkingen en technische termen natuurlijk uit in de tekst. Pas de volgorde van informatie aan voor betere eenvoud, aantrekkelijkheid en leesbaarheid. Probeer geen komma’s of verkleinwoorden te gebruiken."
# # instruction = "Simplify a Dutch paragraph directly into a single, clear, and engaging text suitable for adult readers that speak Dutch as a second language, using words from the 'basiswoordenlijst Amsterdamse kleuters.' Maintain direct quotes, simplify dialogue, explain cultural references, idioms, and technical terms naturally within the text. Adjust the order of information for improved simplicity, engagement, and readability. Attempt to not use any commas or diminutives."
# # instruction = "Vereenvoudig een Nederlandse alinea tot één duidelijke en aantrekkelijke tekst, geschikt voor volwassenen die Nederlands als tweede taal spreken, met woorden uit de 'basiswoordenlijst Amsterdamse kleuters.' Behoud directe citaten, maak dialogen eenvoudiger en leg culturele verwijzingen, uitdrukkingen en technische termen natuurlijk uit in de tekst. Pas de volgorde van informatie aan voor betere eenvoud, aantrekkelijkheid en leesbaarheid. Probeer geen komma’s of verkleinwoorden te gebruiken."
# sentence = random_row['prompt']

# # prompt = '<instruction>' + instruction + '</instruction><simplify>' + sentence + '</simplify>'

# prompt = f'{sentence}'

# # prompt = f'Tune if necessary the following prompt: {instruction}'

# chat = [
#     { "role": "system", "content": system_prompt},  # Adding system instruction
#     { "role": "user", "content": prompt},    # Your existing user prompt
# ]

chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# tokenize the text
input_tokens = tokenizer(chat, return_tensors="pt").to('cpu')

# generate output tokens
output = model.generate(**input_tokens, 
                        max_new_tokens=500)

# decode output tokens into text
output_text = tokenizer.batch_decode(output)

In [13]:
from tqdm import tqdm

In [ ]:
import time
import re
from evaluate import load
from tqdm import tqdm  # For progress bar

df_size = len(df)
sari = load("sari")

# Split dataset into three parts
df_0 = df[:int(df_size*(1/3))]
df_1 = df[int(df_size*(1/3)):int(df_size*(2/3))]
df_2 = df[int(df_size*(2/3)):]

datasets = [df_0, df_1, df_2]
system_prompt = ("Vereenvoudig een Nederlandse alinea tot één duidelijke en aantrekkelijke tekst, "
                 "geschikt voor volwassenen die Nederlands als tweede taal spreken, met woorden uit de "
                 "'basiswoordenlijst Amsterdamse kleuters.' Behoud directe citaten, maak dialogen eenvoudiger "
                 "en leg culturele verwijzingen, uitdrukkingen en technische termen natuurlijk uit in de tekst. "
                 "Pas de volgorde van informatie aan voor betere eenvoud, aantrekkelijkheid en leesbaarheid. "
                 "Probeer geen komma’s of verkleinwoorden te gebruiken.")

sari_scores, sari_scores_avg = [], []
sari_errors = 0

# Start the overall timer
start_time_total = time.time()

for idx, data in enumerate(datasets):
    start_time_data = time.time()  # Timer for each dataset

    print(f"Starting benchmarking on dataset {idx + 1} (size: {len(data)})")

    sari_scores_data = []

    # Use tqdm for a progress bar over rows in the dataset
    for row_idx, row in tqdm(data.iterrows(), desc=f"Processing dataset {idx + 1}"):
        prompt = row['prompt']
        
        template = [
            {"role": "system", "content": system_prompt},  # Adding system instruction
            {"role": "user", "content": prompt},  # Your existing user prompt
        ]
        
        # Tokenize the input and move it to the selected device
        chat = tokenizer.apply_chat_template(template, tokenize=False, add_generation_prompt=True)
        input_tokens = tokenizer(chat, return_tensors="pt").to(device)
        
        # Generate output tokens
        output = model.generate(**input_tokens, max_new_tokens=500)
        
        # Decode output tokens into text
        output_text = tokenizer.batch_decode(output)
        
        # Extract the output using regex
        match = re.search(r'<\|start_of_role\|>assistant<\|end_of_role\|>(.*?)<\|end_of_text\|>', output_text[0])
        if match:
            output = match.group(1).replace('\"', '').replace('\\', '')
        
        # Compute SARI score
        try:
            sari_score = sari.compute(sources=[prompt], predictions=[output], references=[[row['result']]])['sari']
            sari_scores_data.append(sari_score)
        except ValueError as e:
            sari_errors += 1
            print(f'Error: {e}')
            print(f'This originates from the following row: {prompt}')

    # Store the individual dataset scores and average
    sari_scores.append(sari_scores_data)
    sari_scores_avg.append(sum(sari_scores_data) / len(sari_scores_data))

    # Print time taken for each dataset
    elapsed_time_data = time.time() - start_time_data
    print(f"Finished dataset {idx + 1} in {elapsed_time_data:.2f} seconds")

# Final time and results
elapsed_time_total = time.time() - start_time_total
print(f"Processing completed in {elapsed_time_total:.2f} seconds")
print(f"SARI scores averages: {sari_scores_avg}")
print(f"There were {sari_errors} rows that gave an error.")


Starting benchmarking on dataset 1 (size: 89594)


Processing dataset 1: 0it [00:00, ?it/s]

instruction                                             Vereenvoudig: 
prompt               Arthur Dowers is getrouwd en heeft drie kinderen.
result               Arthur Dowers is getrouwd en heeft drie kinderen.
__index_level_0__                                              2486264
Name: 0, dtype: object


NameError: name 'tokenizer' is not defined

In [37]:
output_text

['<|start_of_role|>system<|end_of_role|>Vereenvoudig een Nederlandse alinea tot één duidelijke en aantrekkelijke tekst, geschikt voor volwassenen die Nederlands als tweede taal spreken, met woorden uit de \'basiswoordenlijst Amsterdamse kleuters.\' Behoud directe citaten, maak dialogen eenvoudiger en leg culturele verwijzingen, uitdrukkingen en technische termen natuurlijk uit in de tekst. Pas de volgorde van informatie aan voor betere eenvoud, aantrekkelijkheid en leesbaarheid. Probeer geen komma’s of verkleinwoorden te gebruiken.<|end_of_text|>\n<|start_of_role|>user<|end_of_role|>Montes Abalde leerde al op 10-jarige leeftijd in de muziekschool van de \\"Banda de Música Agrupación Musical Atlántida de Matamá\\" de trombone te bespelen. Vervolgens studeerde hij trombone en compositie aan het \\"Conservatorio de Música de Vigo\\". Hij behaalde zijn diploma\'s voor trombone in oktober 2005 en voor compositie in maart 2007. Later volgde hij ook de muziekopleiding aan de universiteit van 

In [48]:
import re

# Regular expression to extract the text between the markers
match = re.search(r'<\|start_of_role\|>assistant<\|end_of_role\|>(.*?)<\|end_of_text\|>', output_text[0])

if match:
    output = match.group(1).replace('\"', '').replace('\\', '')
    print(output)

Montes Abalde begon met de trombone op 10 jaar oud in de muziekschool van de Banda de Música Agrupación Musical Atlántida de Matamá. Hij studeerde vervolgens trombone en compositie aan het Conservatorio de Música de Vigo en behaalde zijn diploma's in oktober 2005 en maart 2007. Naast zijn band is hij ook lid van het Orquestra Clásica de Vigo.


In [49]:
random_row['result']

'Toen Montes Abalde tien jaar oud was, begon hij trombone te spelen bij de muziekschool van de Banda de Música Agrupación Musical Atlántida de Matamá. Hij ging verder met zijn muziekstudie aan het Conservatorio de Música de Vigo, waar hij leerde hoe hij trombone moest spelen en hoe hij muziek kon componeren. In oktober 2005 kreeg hij zijn diploma voor trombone en in maart 2007 voor compositie. Daarna ging hij naar de universiteit van Vigo voor meer muzieklessen. Montes Abalde speelt niet alleen bij de Banda de Música Agrupación Musical Atlántida de Matamá, maar ook bij het Orquestra Clásica de Vigo.'

In [19]:
# SARI
from evaluate import load
sari = load("sari")

In [18]:
!pip install sacremoses sacrebleu

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 3.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [51]:
sari_score = sari.compute(sources=[sentence], predictions=[output], references=[[random_row['result']]])
sari_score

{'sari': 50.51047164345278}

In [12]:
!pip freeze > requirements.txt